In [2]:
# 训练卷积神经网络
import numpy
import pandas

# order中的key是有序的
from collections import OrderedDict
import re
import os
import io

from sklearn.utils import shuffle

# 评估指标
from sklearn.metrics import roc_auc_score,roc_curve,auc
from scipy import stats

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Embedding
from keras.layers import Conv1D
from keras.utils import np_utils
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD,Adam,RMSprop
from keras.callbacks import EarlyStopping
from keras.models import load_model

from gensim.models import Word2Vec

aa_idx = OrderedDict([
    ('A', 1),
    ('C', 2),
    ('E', 3),
    ('D', 4),
    ('G', 5),
    ('F', 6),
    ('I', 7),
    ('H', 8),
    ('K', 9),
    ('M', 10),
    ('L', 11),
    ('N', 12),
    ('Q', 13),
    ('P', 14),
    ('S', 15),
    ('R', 16),
    ('T', 17),
    ('W', 18),
    ('V', 19),
    ('Y', 20)
])

In [3]:
# 开始训练

# 1、先读入数据
"""
    return test_peptide,test_target,peptide_length,peptide
"""
# 测试集
test_df = pandas.read_csv('train_test_data/test_data/B2705',delim_whitespace=True)
print(test_df.head())

# 通过正则化进行搜索
peptide = re.search(r'[A-Z]\*\d{2}:\d{2}',test_df['Allele'][0]).group()
print(peptide)

print(test_df['Allele'][0])

peptide_length = len(test_df['Peptide_seq'][0])
print(peptide_length)

measurement_type = test_df['Measurement_type'][0]
print(measurement_type)

# 判断是不是binary
if measurement_type.lower() == 'binary':
    test_df['Measurement_value'] = numpy.where(test_df.Measurement_value == 1.0,1,0)
else:
    test_df['Measurement_value'] = numpy.where(test_df.Measurement_value < 500.0,1,0)

# 返回肽和肽对应的标签
test_peptide = test_df.Peptide_seq
test_target = test_df.Measurement_value

test_target = test_target.values

         Date  IEDB_reference       Allele  Peptide_length Measurement_type  \
0  2015-07-31         1029125  HLA-B*27:05               9           binary   
1  2015-07-31         1029125  HLA-B*27:05               9           binary   
2  2015-07-31         1029125  HLA-B*27:05               9           binary   
3  2015-07-31         1029125  HLA-B*27:05               9           binary   
4  2015-07-31         1029125  HLA-B*27:05               9           binary   

  Peptide_seq  Measurement_value  NetMHCpan       SMM      ANN      ARB  \
0   KRGILTLKL                1.0     154.07    257.38    143.0    45.70   
1   KAYKSIVKY                0.0   14040.61  12121.88  11950.0  3691.96   
2   GRLTKHTKR                1.0     633.90    187.31    124.0    31.54   
3   KRYKSIVKR                1.0     102.17    116.30     39.0     7.95   
4   KRYKSIVKL                1.0      55.15    112.35     32.0    21.93   

   SMMPMBEC  IEDB_Consensus  NetMHCcons  PickPocket  
0    229.92         

In [4]:
# 训练集
df_train = pandas.read_csv('train_test_data/train_data/proteins.txt',delim_whitespace=True,header=0)
print(df_train.head())
df_train.columns = ['sequence','HLA','target']

# 创建训练矩阵
df_train = df_train[df_train.HLA == peptide]
print(df_train.head())

df_train = df_train[df_train['sequence'].map(len) == peptide_length]
print(df_train.head())

# 删除未知变量
df_train = df_train[df_train.sequence.str.contains('X') == False]
df_train = df_train[df_train.sequence.str.contains('B') == False]

print(df_train.head())

df_train['target'] = numpy.where(df_train.target == 1,1,0)
seqMatrix = df_train.sequence
targetMatrix = df_train.target
targetMatrix = targetMatrix.values

print('----------------------')
print(seqMatrix)
print('----------------------')
print(targetMatrix)

      Peptide      HLA  BindingCategory
0  AAAAAAAALY  A*29:02                1
1   AAAAALQAK  A*03:01                1
2    AAAAALWL  C*16:01                1
3   AAAAARAAL  B*14:02               -1
4   AAAAEEEEE  A*02:01               -1
      sequence      HLA  target
118  AADFPGIAR  B*27:05      -1
206  AAFLDDNAF  B*27:05      -1
260  AAGLPAIFV  B*27:05      -1
315  AAILKQHKL  B*27:05      -1
353  AAKKKGASL  B*27:05      -1
      sequence      HLA  target
118  AADFPGIAR  B*27:05      -1
206  AAFLDDNAF  B*27:05      -1
260  AAGLPAIFV  B*27:05      -1
315  AAILKQHKL  B*27:05      -1
353  AAKKKGASL  B*27:05      -1
      sequence      HLA  target
118  AADFPGIAR  B*27:05      -1
206  AAFLDDNAF  B*27:05      -1
260  AAGLPAIFV  B*27:05      -1
315  AAILKQHKL  B*27:05      -1
353  AAKKKGASL  B*27:05      -1
----------------------
118       AADFPGIAR
206       AAFLDDNAF
260       AAGLPAIFV
315       AAILKQHKL
353       AAKKKGASL
            ...    
140863    YVYPDNLPR
141059    YYLEKANKI
1

In [24]:
"""
    这里是将原始的肽种的字母全部转换成数字形式，方便计算
    train:featureMatrfix
    test:testMatrix

    return datasets,peptide_length
"""
def aa_integerMapping(peptideSeq):
    """
    map amino acid to its numerical index
    """
    peptideArray = []
    for aa in peptideSeq:
        # 将字母索引对应的数字替换
        peptideArray.append(aa_idx[aa])
    return numpy.asarray(peptideArray)

# 将训练肽序列映射到他们的整数索引
featureMatrix = numpy.empty((0,peptide_length),int)

for num in range(len(seqMatrix)):
    featureMatrix = numpy.append(featureMatrix,[aa_integerMapping(seqMatrix.iloc[num])],axis=0)

print(featureMatrix)

testMatrix = numpy.empty((0,peptide_length),int)
for num in range(len(test_peptide)):
    testMatrix = numpy.append(testMatrix,[aa_integerMapping(test_peptide.iloc[num])],axis=0)

print(len(testMatrix))
print(testMatrix)

# 创建训练和测试的数据集
datasets={}
datasets['X_train'] = featureMatrix
datasets['Y_train'] = targetMatrix
datasets['X_test'] = testMatrix
datasets['Y_test'] = test_target
print(datasets)

[[ 1  1  4 ...  7  1 16]
 [ 1  1  6 ... 12  1  6]
 [ 1  1  5 ...  7  6 19]
 ...
 [20 20 14 ... 19  9 11]
 [20 20 13 ...  8 11  9]
 [20 20 20 ...  3  4 11]]
21
[[ 9 16  5  7 11 17 11  9 11]
 [ 9  1 20  9 15  7 19  9 20]
 [ 5 16 11 17  9  8 17  9 16]
 [ 9 16 20  9 15  7 19  9 16]
 [ 9 16 20  9 15  7 19  9 11]
 [ 5  1 11 17  9  8 17  9  6]
 [16  1 20 13  9 15 17  3 11]
 [ 9 13 20  9 15  7 19  9 20]
 [ 9 16  5  7 11 17 11  9  1]
 [ 5 13  7  4  9 14  7 11  9]
 [16 13 20 13  9 15 17  3 11]
 [16 13 18 11 14  1  5  4  1]
 [ 9 13  5  7 11 17 11  9 20]
 [ 9 16 20  9 15  7 19  9  1]
 [ 9  1  5  7 11 17 11  9 20]
 [ 5 13 11 17  9  8 17  9  6]
 [16  1 18 11 14  1  5  4  1]
 [ 6 13 20 12  5 11  7  8 16]
 [ 5  1  7  4  9 14  7 11  9]
 [ 5 16  7  4  9 14  7 11  1]
 [ 9 16  5  7 11 17 11  9 16]]
{'X_train': array([[ 1,  1,  4, ...,  7,  1, 16],
       [ 1,  1,  6, ..., 12,  1,  6],
       [ 1,  1,  5, ...,  7,  6, 19],
       ...,
       [20, 20, 14, ..., 19,  9, 11],
       [20, 20, 13, ...,  8, 11,  

In [34]:
# CNN parameters
# ceil向上取整
batch_size = int(numpy.ceil(len(datasets['X_train']) / 100))
epochs = 100
nb_filter = 32
filter_length = 7
dropout = 0.25
lr = 0.004

In [44]:
# 加载word2vec中学习的分布式表示的HLA-Vec
"""
    这里直接测试生成的.model和无.model的结果一样的
"""
hla_vec_obj = Word2Vec.load(os.path.join('HLA-Vec_embedding','HLA-Vec_Object'))
print(hla_vec_obj)
hla_vec_embd = hla_vec_obj.wv
embd_shape = hla_vec_embd.syn0.shape
print(embd_shape)

# 随机生成一组数据：均匀分布
embedding_weights = numpy.random.rand(embd_shape[0]+1,embd_shape[1])
for key in aa_idx.keys():
    embedding_weights[aa_idx[key],:] = hla_vec_embd[key]
embedded_dim = embd_shape[1]

Word2Vec(vocab=20, size=15, alpha=0.025)
(20, 15)


In [46]:
i = 0
while True:
    model = None
    model = Sequential()

    # 嵌入模块
    model.add(Embedding(input_dim=len(aa_idx)+1,output_dim=embedded_dim,weights=[embedding_weights],input_length=peptide_length,trainable=True))

    # 卷积模块1
    model.add(Conv1D(nb_filter,filter_length,padding='same',kernel_initializer='glorot_normal'))
    model.add(LeakyReLU(.3))
    model.add(Dropout(dropout))

    # 卷积模块2
    # 1和2是相同的模块
    model.add(Conv1D(nb_filter, filter_length, padding="same", kernel_initializer="glorot_normal"))
    model.add(LeakyReLU(.3))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(nb_filter*peptide_length))
    model.add(Activation('sigmoid'))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=lr),
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='loss',patience=2,verbose=1,mode='auto')
    mod = model.fit(datasets['X_train'],datasets['Y_train'],batch_size==batch_size,epochs=epochs,verbose=1,callbacks=[earlyStopping],shuffle=True,validation_data=(datasets['X_test'],datasets['Y_test']))
    modLoss = mod.history['loss']

    # 检查以确保优化没有发散
    if ~numpy.isnan(modLoss[-1]):
        #保存模型
        model.save(os.path.join('HLA-CNN_models','hla_cnn_model_'+str(i)+'.hdf5'))

        i += 1
        if i>4:
            break


Epoch 1/100
2436/2436 [==============================] - 7s 3ms/step - loss: 0.6837 - accuracy: 0.6507 - val_loss: 0.6367 - val_accuracy: 0.6667